In [1]:
import sys

In [2]:
sys.path.append("/Users/diegoesparza/CS_Ventures/current_projects/chess_engine_using_probabilities/my_engine/chess-sauce")

In [8]:
import my_engine
import chess
import stockfish
import random
from line_profiler import LineProfiler
import pandas as pd
from enum import Enum

In [4]:
sf = stockfish.Stockfish(path="/Users/diegoesparza/Downloads/stockfish/stockfish-macos-x86-64")
sf.resume_full_strength()

In [11]:
# if second best piece to move can't make a move nearly as good as best move,
# probably not stable
# 
# if best non-capture/non-check move isn't nearly as good as best move, probably
# not stable.
#
# board piece eval at end should be within -200 200
# centipawn eval should be within -200 200 (this we'll track as we go along)
class Stability(Enum):
    EVAL_UNSTABLE = "EVAL_UNSTABLE"
    PIECE_VAL_UNSTABLE = "PIECE_VAL_UNSTABLE"
    CAPTURE_UNSTABLE = "CAPTURE_UNSTABLE"
    FORCED_MOVES_UNSTABLE = "FORCED_MOVES_UNSTABLE"
    STABLE = "STABLE"
    
def position_is_stable(b, sf) -> Stability:
    # do this first because if it hits, we give up on the position.
    eval = sf.get_evaluation(searchtime=20)
    if eval["type"] != "cp" or abs(eval["value"]) > 200:
        return Stability.EVAL_UNSTABLE

    # assume sf is set to b's position
    score, _ = my_engine.eval_piece_vals.eval_piece_vals(b)
    if abs(score) > 200:
        return Stability.PIECE_VAL_UNSTABLE
   
   
    moves = sf.get_top_moves(7, num_nodes=10000)
    for move in moves:
        move["Move"] = chess.Move.from_uci(move["Move"])
    top_centipawn = moves[0]["Centipawn"]
    top_origin = moves[0]["Move"].from_square
    mult = 1 if b.turn else -1
   
    # check origins, captures
    must_capture = True
    must_move_specific_piece = True
    for move in moves:
        centipawn = move["Centipawn"]
        if mult * centipawn + 200 < mult * top_centipawn:
            break
           
        if not b.is_capture(move["Move"]):
            must_capture = False
        if move["Move"].from_square != top_origin:
            must_move_specific_piece = False
            
        if not (must_capture or must_move_specific_piece):
            break
    if must_capture:
        return Stability.CAPTURE_UNSTABLE
    elif must_move_specific_piece:
        return Stability.FORCED_MOVES_UNSTABLE
    else:
        return Stability.STABLE


def gen_positions_from(b, sf, positions_to_generate, depth):
    fen = b.fen()
    sf.set_fen_position(fen)
    if depth == 0:
        if positions_to_generate != 1:
            raise ValueError(positions_to_generate)
        stability = position_is_stable(b, sf)
        if stability == Stability.STABLE:
            print(f"stable: {fen}")
            return None
        elif stability == Stability.EVAL_UNSTABLE:
            print(f"eval unstable: {fen}")
            return None

        depth = 1

    moves = sf.get_top_moves(min(7, positions_to_generate), num_nodes=10000)
    moves = [move for move in moves if abs(move["Centipawn"]) < 500]
    if len(moves) == 0:
        print(f"skewed: {fen}")
        return None
            
    moves_to_explore = [chess.Move.from_uci(move["Move"]) for move in moves]
    next_depth = depth-1
    moves = list(b.legal_moves)

    pos_per = positions_to_generate // len(moves_to_explore)
    rem = positions_to_generate - pos_per * len(moves_to_explore)
    for idx, move in enumerate(moves_to_explore):
        next_positions_to_generate = pos_per
        if idx < rem:
            next_positions_to_generate += 1
        b.push(move)
        gen_positions_from(b, sf, next_positions_to_generate, next_depth)
        b.pop()
        
def gen_positions(sf, opening_fens, positions_to_generate, depth):
    if positions_to_generate < len(opening_fens):
        opening_fens = random.sample(opening_fens, positions_to_generate)
        
    pos_per = positions_to_generate // len(opening_fens)
    rem = positions_to_generate - pos_per * len(opening_fens)
    for idx, fen in enumerate(opening_fens):
        next_positions_to_generate = pos_per
        if idx < rem:
            next_positions_to_generate += 1
        b = chess.Board(fen)
        try:
            gen_positions_from(b, sf, next_positions_to_generate, depth)
        except ValueError as e:
            print(f"Error from {fen}: {e}")

In [12]:
positions_to_generate = 10
depth = 8
random.seed(0)

lp = LineProfiler()
lp.add_function(gen_positions_from)
lp.add_function(gen_positions)
lp.add_function(position_is_stable)
opening_fens = !cat "/Users/diegoesparza/CS_Ventures/current_projects/chess_engine_using_probabilities/kaggle_fens/strong_openings" # type: ignore
lp.runcall(gen_positions, sf, opening_fens, positions_to_generate, depth)
lp.print_stats()

stable: r2q1rk1/4b1pp/2p5/p1PpP3/3Qp3/8/PP3PPP/R1B2RK1 w - - 0 17
stable: r2q1r2/ppp1nppk/3p1n1p/4p3/P1BPP1Q1/2N1P2P/1PP3P1/R4RK1 w - - 3 15
piece evals: -265.125
stable: r2q1rk1/p1ppbppp/1p2p3/n3P3/3P1P1P/P1Q1BN2/1PP3P1/2K2R1R b - - 0 15
stable: 2rq1r1k/1b1pnpbp/p5p1/1p1Np3/1P1nP3/P1N5/BB3PPP/R2Q1RK1 w - - 3 17
piece evals: -293.375
stable: r1bq1rk1/2p3p1/1pn1pp2/3pP3/3P3Q/P1P5/5PPP/R1B1KB1R b KQ - 0 15
stable: r3kb1r/pp3ppp/8/8/2Bn2b1/6P1/PPPB1PP1/R3K1NR w KQkq - 1 13
stable: rnbq2k1/4bp1p/2pp3B/1p2p2p/p3P3/P1NP1N2/BPP2PPP/R3K2R b KQ - 1 15
stable: r2r2k1/1p1bqppp/1p2pn2/2p1N3/1P1n4/P2B4/1BPQ1PPP/R2R2K1 w - - 0 17
piece evals: -473.5
piece evals: -377.5
piece evals: -373.0
stable: r1bq1Bk1/p4p1p/2p1pbp1/4P3/4Q3/3B4/PPP2PPP/R5K1 b - - 0 16
stable: r2q1rk1/3n1pp1/b3pn1p/p2p4/R2P1B2/1QPBPN2/5PPP/R5K1 w - - 2 17
Timer unit: 1e-09 s

Total time: 0.59356 s
File: /var/folders/j2/1qmnvcts25x7hrnmfmyczwg40000gn/T/ipykernel_28466/3089066167.py
Function: position_is_stable at line 16

Line #   

In [22]:
sf.set_fen_position("Bn1qbrkb/p2n1p1p/6p1/3p4/Q2P3P/4PN2/PP2KPP1/R1B4R b - - 0 14", send_ucinewgame_token=True)
for i in range(10):
    print(sf.get_top_moves(5, num_nodes=10000))

[{'Move': 'd7c5', 'Centipawn': -147, 'Mate': None}, {'Move': 'd7b6', 'Centipawn': -189, 'Mate': None}, {'Move': 'd7e5', 'Centipawn': -151, 'Mate': None}, {'Move': 'h8f6', 'Centipawn': -205, 'Mate': None}, {'Move': 'd8c8', 'Centipawn': -211, 'Mate': None}]
[{'Move': 'd7c5', 'Centipawn': -120, 'Mate': None}, {'Move': 'd7b6', 'Centipawn': -193, 'Mate': None}, {'Move': 'd8c7', 'Centipawn': -193, 'Mate': None}, {'Move': 'h8f6', 'Centipawn': -217, 'Mate': None}, {'Move': 'd8c8', 'Centipawn': -222, 'Mate': None}]
[{'Move': 'd7c5', 'Centipawn': -92, 'Mate': None}, {'Move': 'a7a6', 'Centipawn': -173, 'Mate': None}, {'Move': 'd7b6', 'Centipawn': -190, 'Mate': None}, {'Move': 'h8g7', 'Centipawn': -193, 'Mate': None}, {'Move': 'd8c7', 'Centipawn': -201, 'Mate': None}]
[{'Move': 'd7c5', 'Centipawn': -69, 'Mate': None}, {'Move': 'd7b6', 'Centipawn': -191, 'Mate': None}, {'Move': 'a7a6', 'Centipawn': -208, 'Mate': None}, {'Move': 'h8f6', 'Centipawn': -208, 'Mate': None}, {'Move': 'h8g7', 'Centipawn':